3.Merchant features: extracted from the training set 1, training set 2 and test set feature interval
total_sales # Show the number of sales per item
sales_use_coupon # Show items that have been consumed with coupons
total_coupon # The total number of coupons showing the item
coupon_rate = sales_use_coupon/total_sales # Percentage of coupons used in selling merchandise
merchant_coupon_transfer_rate = sales_use_coupon/total_coupon # Coupon usage rate

In [2]:
import os, sys, pickle
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import SGDClassifier, LogisticRegression

dfoff = pd.read_csv('ccf_offline_stage1_train.csv')
dftest = pd.read_csv('ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('ccf_online_stage1_train.csv')
#print(dfoff.head)
#print(dftest.head)
#print(dfon.head)


In [6]:
"""
total_sales # Show the number of sales per item
sales_use_coupon # Show items that have been consumed with coupons
total_coupon # The total number of coupons showing the item
coupon_rate = sales_use_coupon/total_sales # Percentage of coupons used in selling merchandise
merchant_coupon_transfer_rate = sales_use_coupon/total_coupon # Coupon usage rate
"""
def get_off_Merchant_features(dfoff):
    Merchant_features=pd.DataFrame()
    Merchant_features['Merchant_id']=dfoff['Merchant_id'].unique()
    #print(Merchant_features.head(5))
    
    # 商家所有商品的销售总量（用+不用优惠券）off_total_sales
    M1=dfoff[['Merchant_id','Date']]
    M1=M1.dropna()
    temp1 = M1.groupby('Merchant_id').size().reset_index(name='off_total_sales')
    Merchant_features= pd.merge(Merchant_features, temp1, how='left', on='Merchant_id')
    
    # 商家被领取的优惠券数目off_total_coupon
    M2=dfoff[['Merchant_id','Coupon_id']]
    M2=M2.dropna()
    temp2 = M2.groupby('Merchant_id').size().reset_index(name='off_total_coupon')
    Merchant_features= pd.merge(Merchant_features, temp2, how='left', on='Merchant_id')
    
    
    # 商家发放的优惠券被使用的数量off_sales_use_coupon
    M3 = dfoff[['Merchant_id','Coupon_id','Date']]
    M3=M3.dropna()
    temp3 = M3.groupby('Merchant_id').size().reset_index(name='off_sales_use_coupon')
    Merchant_features= pd.merge(Merchant_features, temp3, how='left', on='Merchant_id')
    
    # 商家发行的所有优惠券种类数目 off_total_coupon_type
    M4 = dfoff.groupby(['Merchant_id', 'Coupon_id']).size()
    M4 = M4.groupby('Merchant_id').size().reset_index(name='off_total_coupon_type')
    Merchant_features = pd.merge(Merchant_features, M4, how='left', on='Merchant_id')
    
    # 商家被多少不同用户领取的数目 off_total_user
    M5 = dfoff.groupby(['Merchant_id', 'User_id']).size()
    M5 = M5.groupby('Merchant_id').size().reset_index(name='off_total_user')
    Merchant_features = pd.merge(Merchant_features, M5, how='left', on='Merchant_id')
    
    #coupon_rate = sales_use_coupon/total_sales
    #merchant_coupon_transfer_rate = sales_use_coupon/total_coupon
    Merchant_features=Merchant_features.fillna(0)
    Merchant_features['off_coupon_rate']=Merchant_features['off_sales_use_coupon']/Merchant_features['off_total_sales']
    Merchant_features['merchant_coupon_not_use']=Merchant_features['off_total_coupon']-Merchant_features['off_sales_use_coupon']
    Merchant_features['off_merchant_coupon_transfer_rate']=Merchant_features['off_sales_use_coupon']/Merchant_features['off_total_coupon']
    
    #add to dfoff
    dfoff = pd.merge(dfoff, Merchant_features, how='left')
    return dfoff
"""
def get_on_Merchant_features(dfon):
    Merchant_features2=pd.DataFrame()
    Merchant_features2['Merchant_id']=dfon['Merchant_id'].unique()
    #print(Merchant_features.head(5))
    
    # 商家所有商品的销售总量（用+不用优惠券）on_total_sales
    temp=dfon[['Merchant_id','Date']]
    temp=temp.dropna()
    temp1 = temp.groupby('Merchant_id').size().reset_index(name='on_total_sales')
    Merchant_features2= pd.merge(Merchant_features2, temp1, how='left', on='Merchant_id')
    
    # 商家被领取的优惠券数目on_total_coupon
    temp=dfon[['Merchant_id','Coupon_id']]
    temp=temp.dropna()
    temp1 = temp.groupby('Merchant_id').size().reset_index(name='on_total_coupon')
    Merchant_features2= pd.merge(Merchant_features2, temp1, how='left', on='Merchant_id')
    
    
    # 商家发放的优惠券被使用的数量on_sales_use_coupon
    temp = dfon[['Merchant_id','Coupon_id','Date']]
    temp=temp.dropna()
    temp1 = temp.groupby('Merchant_id').size().reset_index(name='on_sales_use_coupon')
    Merchant_features2= pd.merge(Merchant_features2, temp1, how='left', on='Merchant_id')
    
    # 商家发行的所有优惠券种类数目 on_total_coupon_type
    temp = dfon.groupby(['Merchant_id', 'Coupon_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='on_total_coupon_type')
    Merchant_features2 = pd.merge(Merchant_features2, temp, how='left', on='Merchant_id')
    
    # 商家被多少不同用户领取的数目 on_total_user
    temp = dfon.groupby(['Merchant_id', 'User_id']).size()
    temp = temp.groupby('Merchant_id').size().reset_index(name='on_total_user')
    Merchant_features2 = pd.merge(Merchant_features2, temp, how='left', on='Merchant_id')
    
    #coupon_rate = sales_use_coupon/total_sales
    #merchant_coupon_transfer_rate = sales_use_coupon/total_coupon
    Merchant_features2=Merchant_features2.fillna(0)
    Merchant_features2['on_coupon_rate']=Merchant_features2['on_sales_use_coupon']/Merchant_features2['on_total_sales']
    Merchant_features2['on_merchant_coupon_transfer_rate']=Merchant_features2['on_sales_use_coupon']/Merchant_features2['on_total_coupon']
    return Merchant_features2
"""

    
if __name__ == "__main__":
    Merchant_features,dfoff=get_off_Merchant_features(dfoff)
    print(dfoff.head(5))
    #Merchant_features.to_csv("offline_Merchant_features.csv",index=None)
    #Merchant_features2=get_on_Merchant_features(dfon)
    print(Merchant_features.head(5))
    
 

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632        NaN           NaN       0.0            NaN   
1  1439408         4663    11002.0        150:20       1.0     20160528.0   
2  1439408         2632     8591.0          20:1       0.0     20160217.0   
3  1439408         2632     1078.0          20:1       0.0     20160319.0   
4  1439408         2632     8591.0          20:1       0.0     20160613.0   

         Date  off_total_sales  off_total_coupon  off_sales_use_coupon  \
0  20160217.0             21.0              43.0                   4.0   
1         NaN           1434.0           15191.0                  76.0   
2         NaN             21.0              43.0                   4.0   
3         NaN             21.0              43.0                   4.0   
4         NaN             21.0              43.0                   4.0   

   off_total_coupon_type  off_total_user  off_coupon_rate  \
0                    2.0       